In [3]:
import pandas as pd

url = 'https://raw.githubusercontent.com/DyanelLancea/Airline-Review-Sentiement-Analysis/refs/heads/master/airlines_review.csv'
df = pd.read_csv(url, index_col=0)
print(df.head(5))

              Name  Location Date Published  \
0  Harry Aronowicz      2025     2025-08-31   
1  Harry Aronowicz      2025     2025-08-31   
2     Sreina Ilyas      2025     2025-08-31   
3        W Johnson      2025     2025-08-28   
4         J Kamara      2025     2025-08-25   

                                        Text Content       Seat Type  \
0  ✅ Trip Verified |   Excellent flight. Super cr...  Boeing 777-200   
1  ✅ Trip Verified |  Flight left and arrived on ...            A319   
2  Not Verified |    My friend and I were helped ...            A320   
3  ✅ Trip Verified | As a regular flier in this r...         A320neo   
4  ✅ Trip Verified |   I've been a regular travel...        Business   

   Seat Comfort  Cabin Staff Service  Food & Beverages  \
0           NaN                  NaN               NaN   
1           NaN                  NaN               NaN   
2           NaN                  NaN               NaN   
3           NaN                  NaN               N